In [1]:
import paramiko
import pandas as pd
import numpy as np
import os
from collections import defaultdict, namedtuple
from label_extractor import label_extractor

In [4]:
class RemoteSrc:
    REMOTE_PATH = "/sgd-data/data/stock/"
    TEMP = "/home/yby/YBY/CNN/backtest_temp/"

    def __init__(self):
        self._client = paramiko.Transport(("192.168.1.147", 22))
        self._client.connect(username="sgd", password="sgd123")
        self._SFTP = paramiko.SFTPClient.from_transport(self._client)
        if not os.path.exists(self.TEMP):
            os.mkdir(self.TEMP)

        self.dict_stocksPerDay = defaultdict(list)

    def get_raw_bars(self, ticker, date):

        local_path = f"{self.TEMP}{ticker}_{date}.csv.gz"

        if not os.path.exists(local_path):
            files_currentDay = self._SFTP.listdir(f"{self.REMOTE_PATH}{date}/tick_csv/")
            if date in self.dict_stocksPerDay.keys():
                stocks_currentDay = self.dict_stocksPerDay[date]
            else:
                stocks_currentDay = [s[:6] for s in files_currentDay]

            file_idx = stocks_currentDay.index(ticker)

            self._SFTP.get(remotepath=f"{self.REMOTE_PATH}{date}/tick_csv/{files_currentDay[file_idx]}",
                           localpath=local_path)

        data = pd.read_csv(local_path)
        data['server_time'] = pd.to_datetime(data.server_time)
        data['local_time'] = data['server_time']
        data['time'] = data.apply(lambda x: str(x['server_time'].time()), axis = 1)

        return data

In [9]:
DATA_PATH = "/home/yby/YBY/Data_labels"

In [7]:
class Predict:
    def __init__(self):
        self.remote_server = RemoteSrc()


    def specific_stock(self, stock_id, date):

        data = pd.read_pickle(f"{DATA_PATH}/{stock_id}/{date}.pkl").fillna('0')
        raw_data = self.remote_server.get_raw_bars(ticker = stock_id, date = date).set_index("time")
        data.set_index("time", inplace = True)
        
        res = pd.merge(data, raw_data, how = "inner", left_index = True, right_index = True)

        return res



def query(stock_id, date):
    SAVING_PATH = '../Backtest/'
    if not os.path.exists(SAVING_PATH):
        os.makedirs(SAVING_PATH)
    predict = Predict()

    stock_bars:pd.DataFrame = predict.specific_stock(stock_id=stock_id, date=date)
    return stock_bars

In [112]:
df = query("603290", "20211101")

In [171]:
df.reset_index(inplace = True)
df["time"] = pd.to_datetime(df["time"])


In [174]:
df.set_index("time", inplace = True)

In [175]:
data = df[["vwp", "price", "volume"]]

In [176]:
data["net_vol"] = data["volume"].diff()

/tmp/ipykernel_1481328/11271596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["net_vol"] = data["volume"].diff()


In [ ]:
def test_deal_diff(window):
    data = df[["vwp", "price", "volume"]]
    data["net_volume"] = data.volume.diff()
    data.iloc[0, 3] = data.iloc[0, 2]
    data["deal"] = data["net_vol"] * data["price"]
    data["vwp_hat"] = data["deal"].rolling(window, closed = "right").sum() / data["net_vol"].rolling(window, closed = "right").sum()
    data["deal_diff"] = data["vwp_hat"] - data["]

In [240]:
data["deal"] = data["net_vol"] * data["price"]
data["vwp_hat"] = data["deal"].rolling(200, closed = "right").sum() / data["net_vol"].rolling(200, closed = "right").sum()

/tmp/ipykernel_1481328/572854824.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["deal"] = data["net_vol"] * data["price"]
/tmp/ipykernel_1481328/572854824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["vwp_hat"] = data["deal"].rolling(200, closed = "right").sum() / data["net_vol"].rolling(200, closed = "right").sum()


In [243]:
data.mean()["deal_diff"]

-41.99051980716713

In [222]:
data["vwp_deal"] = data["vwp"] * data["net_vol"]

/tmp/ipykernel_1481328/951073025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["vwp_deal"] = data["vwp"] * data["net_vol"]


In [244]:
data

,vwp,price,volume,net_vol,deal,vwp_hat,weight,vwp_deal,deal_diff
time,,,,,,,,,
2022-06-23 09:30:03,401.096176,400.965,12600,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-23 09:30:06,401.047632,401.450,16400,3800.0,1525510.0,NaN,3803.812515,1523981.0,1529.0
2022-06-23 09:30:09,401.244853,401.015,23200,6800.0,2726902.0,NaN,6796.104623,2728465.0,-1563.0
2022-06-23 09:30:12,400.950000,400.525,26300,3100.0,1241627.5,NaN,3096.714054,1242945.0,-1317.5
2022-06-23 09:30:15,400.215455,400.035,27400,1100.0,440038.5,NaN,1099.504017,440237.0,-198.5
...,...,...,...,...,...,...,...,...,...
2022-06-23 14:56:45,437.690000,437.495,4285050,600.0,262497.0,438.120033,599.732688,262614.0,-117.0
2022-06-23 14:56:48,437.695000,437.745,4285250,200.0,87549.0,438.116181,200.022847,87539.0,10.0
2022-06-23 14:56:51,437.495000,437.745,4285650,400.0,175098.0,438.111596,400.228574,174998.0,100.0


In [233]:
data["deal_diff"].describe()

count      4771.000000
mean        -41.990520
std        1769.025557
min     -105414.500000
25%         -21.500000
50%           0.000000
75%          20.000000
max       18056.000000
Name: deal_diff, dtype: float64

In [ ]:
data["diff"]